In [1]:
# UrbanClassifier Toolkit

In [2]:
# Import Libraries

In [3]:
import gdal
import math
import numpy as np
import json

In [4]:
# Open TIFF File, and convert raster to multidimensional array format - convert into a matrix that is stored as a multidimensional array
# NOTE : Matrix is very large

In [5]:
src = gdal.Open('gpw_v4_population_count_rev11_2020_30_sec.tif')
myarray = np.array(src.GetRasterBand(1).ReadAsArray())

In [6]:
# GeoTrans takes the Header Data from the Raster File to make proper Calculations

In [7]:
geoTrans = src.GetGeoTransform()

In [8]:
# geoTrans is stored as the following : (-180.0, 0.00833333333333333, 0.0, 89.99999999999991, 0.0, -0.00833333333333333)

In [9]:
# Coordinates Conversion Function : 

In [10]:
def world2Pixel(geoMatrix, x, y):
    
    # Uses a gdal geomatrix ( gdal.GetGeoTransform() ) to calculate the pixel location of a geospatial coordinate
    
    ulX = geoMatrix[0]
    ulY = geoMatrix[3]
    xDist = geoMatrix[1]
    yDist = geoMatrix[5]
    rtnX = geoMatrix[2]
    trnY = geoMatrix[4]
    pixel = int((x - ulX) / xDist)
    line = int((ulY - y) / xDist)
    
    return (pixel, line)

In [11]:
# Helper Function to force odd conversion : 

In [12]:
def toOdd(n):
    if (n % 2) == 0:
        return n + 1
    else:
        return n

In [13]:
# Radius Iterative Function : 

In [50]:
def radius(arr, pixel, line, n):
    
    # Takes as :
    # arr = input matrix from TIFF File
    # pixel = specific pixel from world2Pixel Function
    # line = specific line from world2Pixel Function
    # n = a numeric value, interpreted by the function as an n by n radius in Km Squared 
    # NOTE: n is automatically converted to an odd number if input is even
    
    r = toOdd(n)
    
    # toOdd returns ( n + 1 ) if n is even, message to indicate this is printed : 
    
    # TODO: boolean to display conversion ?
    
    # print("Attention : Input n with value " + str(n) + " was converted to value " + str(r) + " to preform more accurate calculation")
    
    # Calculate distance value to subtract from starting line and cell
    
    dist = math.floor(r / 2)
    
    # Determine starting points
    
    start_pixel = pixel - dist
    start_line = line - dist
    
    # Simply initialize sum variable `pop_sum` to 0
    
    pop_sum = 0
    
    # pop_sum is the total sum of returned population values for each Km squared cell present in the specified radius
    
    for i in range(r):
        
        # Iteratively extracting population data from each cell, and adding it to pop_sum
        # i refers to index of cell
        
        for j in range(r):
            
            # j refers to index of line
            # adds cell i for each line j through r, for each iteration called by parent loop
            
            pop_sum += arr[start_line + j][start_pixel + i]
               
                
    return pop_sum

In [15]:
# Function which returns Total Population for given radius :

In [69]:
def popDensityTotal(arr, x, y, r):
    
    # x = longitude
    # y = latitude
    # arr = Matrix created from TIFF File 
    # r = specified radius
    
    try:
    
        pixel = world2Pixel(geoTrans, x, y)[0]
        line = world2Pixel(geoTrans, x, y)[1]
        return radius(myarray, pixel, line, r)
    
    except:
        
        return "Error"

In [17]:
# Function which returns Average Population for given radius :

In [63]:
def popDensityAverage(arr, x, y, r):
    
    try: 
    
        # Gets total by calling popDensityTotal

        total = popDensityTotal(arr, x, y, r)

        # Returns total / r * r to account for total divided by number of cells in radius

        cells = math.pow(toOdd(r), 2)
        avg = total / cells
    
        return avg
    
    except: 'Error'

In [19]:
# CLASSIFICATION FUNCTIONS : 

In [20]:
# Both Classification Functions take in as input : (bound, arr, x , y, r) 

# bound = specified threshold, numerical value, to classify as `Urban`
# arr = Matrix extracted from TIFF File
# x = longitude
# y = latitude
# r = radius
    
    

In [48]:
def urbanByAverage(bound, arr, x , y, r): 
    
    # Call popDensityAverage
    
    try:
    
        avg = popDensityAverage(arr, x, y, r)

        # If the avg population is greater than or equal to the specified bound, return true

        if avg >= bound: return True 

        else : return False 
        
    except: 
        return "Error"

In [22]:
def urbanByTotal(bound, arr, x , y, r): 
    
    # Call popDensityTotal
    
    total = popDensityTotal(arr, x, y, r)

    # If the total population is greater than or equal to the specified bound, return true
    
    if total >= bound: return True 
    
    else : return False 

In [23]:
# Additional Function :

In [24]:
# Returns a specific population density for a given pair of coordinates
# Converts coordinates to a reference line and cell, and looks up said line and cell in the Matrix ( represented as a multidimensional array )

In [25]:
def popDensity(arr, x, y):
    pixel = world2Pixel(geoTrans, x, y)[0]
    line = world2Pixel(geoTrans, x, y)[1]
    return arr[line][pixel]

In [26]:
# EXAMPLES :

In [27]:
popDensity(myarray,-79.347015, 43.651070 )

688.37634

In [28]:
# popDensityTotal with radius of 111 ( one-hundred-and-eleven ) -- ( 111 x 111 Km Squared )

popDensityTotal(myarray, -79.347015, 43.651070, 111)

Attention : Input n with value 111 was converted to value 111 to preform more accurate calculation


6559703.393304814

In [29]:
# popDensityAverage with radius of 11 ( eleven ) -- ( 11 x 11 Km Squared )

popDensityAverage(myarray, -79.347015, 43.651070, 11)

Attention : Input n with value 11 was converted to value 11 to preform more accurate calculation


2918.7404481828953

In [30]:
# urbanByTotal with radius of 111 ( one-hundred-and-eleven ) -- ( 111 x 111 Km Squared ), and threshold of 7 million people
# Return is False, since popDensityTotal returns `6559703.393304814` as displayed above 

urbanByTotal(7000000, myarray, -79.347015, 43.651070, 111)

Attention : Input n with value 111 was converted to value 111 to preform more accurate calculation


False

In [31]:
# urbanByAverage with radius of 11 ( eleven ) -- ( 11 x 11 Km Squared ), and threshold of 2000 people
# Return is True, since popDensityTotal returns `2918.7404481828953` as displayed above 

urbanByAverage(2000, myarray, -79.347015, 43.651070, 11)

Attention : Input n with value 11 was converted to value 11 to preform more accurate calculation


True

In [32]:
# Testing

In [33]:
def readFile(filepath): 
    
    with open(filepath, 'r') as f:
        stories = json.load(f)
        
    return list(stories)

In [34]:
stories = readFile('stories.json')

In [35]:
def extractCoordinates(stories):
    
    result = []
    
    for story in stories:
        
        
        obj = {
            'title' : story['title'],
            'longitude' : story['longitude'],
            'latitude' : story['latitude']
        }
        
        result.append(obj)
        
        
    return result

In [36]:
coordinates = extractCoordinates(stories)

In [40]:
# Example -> Coordinate Object Format

In [39]:
coordinates[0]

{'title': 'Books Around the Block',
 'longitude': -93.2762122742247,
 'latitude': 44.992024098754}

In [82]:
def test(stories, bound, arr, r, by_average):
    
    coordinates = extractCoordinates(stories)
    
    if by_average:

        for coordinate in coordinates: 

            x = coordinate['longitude']
            y = coordinate['latitude']

            coordinate['urban'] = urbanByAverage(bound, arr, x, y, r)
            coordinate['bound'] = str(bound) + ' People'
            coordinate['radius'] = str(math.pow(toOdd(r), 2)) + ' Km Squared'
            coordinate['average'] = str(popDensityAverage(arr, x, y, r)) + ' / Km Squared'
            
            
    else:

        for coordinate in coordinates: 

            x = coordinate['longitude']
            y = coordinate['latitude']

            coordinate['urban'] = urbanByAverage(bound, arr, x, y, r)
            coordinate['bound'] = str(bound) + ' People'
            coordinate['radius'] = str(math.pow(toOdd(r), 2)) + ' Km Squared'
            coordinate['total'] = str(popDensityTotal(arr, x, y, r)) + ' Total'
        

    return coordinates

In [76]:
test = test(stories, 100000, myarray, 20, False)

In [77]:
test[0]

{'title': 'Books Around the Block',
 'longitude': -93.2762122742247,
 'latitude': 44.992024098754,
 'urban': False,
 'bound': '100000 People',
 'radius': '441.0 Km Squared',
 'total': '544424.9896454811 Total'}

In [85]:
def extract(stories, bound, arr, r, by_average):
    
    test_set = test(stories, bound, arr, r, by_average)
    
    with open('Tests/bound=' + 'by_average=' + str(by_average) + '&' + str(bound) + '&radius=' + str(math.pow(toOdd(r), 2)) + '.json', 'w') as json_file:
        json.dump(test_set, json_file)
    
    return

In [87]:
extract(stories, 100000, myarray, 20, False)